Funções para cada módulo da interface

In [2]:
import json
import plotly.express as px
import pandas as pd

In [ ]:
def minutos_segundos(lista):
    segundos = (int(lista[0]) * 3600) + int(lista[1] * 60) + int(lista[2])
    return segundos

---

Rupturas da linha de defesa

In [ ]:
def infos_ruptura(json, times_id):
    dict_rupturas_time_x = {}
    dict_rupturas_time_y = {}

    for id in times_id:
        n_ruptura = 0
        rupturas = json["time"][f"{id}"]["rupturas"]

        for ruptura in rupturas:
            n_ruptura += 1
            instante_ruptura = ruptura["instante_ruptura"]

            # Verifica em qual tempo do jogo a ruptura ocorreu
            lista_instante_ruptura = instante_ruptura.split(":")
            segundos = minutos_segundos(lista_instante_ruptura)
            if segundos <= 2700:
                tempo = "1º T"
            else:
                tempo = "2º T"

            # Informações sobre a ruptura
            infos = {
                "logo": json["time"][f"{id}"]["logo"],
                "instante_ruptura": ruptura["instante_ruptura"],
                "tempo": tempo,
                "desfecho": ruptura["desfecho"],
                "zona": ruptura["zona_defesa"],
                "posse_bola": ruptura["nome_jogador_posse_bola"],
                "jogador_em_ruptura": ruptura["nome_jogador_ruptura"],
                "jogadores_defesa": ruptura["nomes_jogadores_defesa"],
            }

            # Adiciona as informações no dicionário
            if id == 1:
                dict_rupturas_time_x[f"ruptura_{n_ruptura}"] = infos
            else:
                dict_rupturas_time_y[f"ruptura_{n_ruptura}"] = infos

    return dict_rupturas_time_x, dict_rupturas_time_y

---

Top 5 Jogadores com mais rupturas

In [ ]:
def top_5_rupturas(json, time_id):
    dict_top_5 = {}
    top_5 = json["time"][f"{time_id}"]["top_5"]

    for jogador in top_5:
        nome = jogador["nome"]
        n_rupturas = jogador["n_rupturas"]

        dict_top_5[nome] = n_rupturas
    
    return dict_top_5    

---

Gráfico de desfechos

In [3]:
def grafico_desfechos_quebra_linha(json, time_id):
    desfechos = json["time"][f"{time_id}"]["desfechos"]
    desfecho = []
    n_rupturas = []

    for tipo, quantidade in desfechos.items():
        desfecho.append(tipo)
        n_rupturas.append(quantidade)

    # Fazendo o gráfico com o Plotly
    df = pd.DataFrame(dict(
        desfecho=desfecho,
        n_rupturas=n_rupturas
        ))
    
    fig = px.pie(df, values='n_rupturas', names='desfecho', title='Desfechos das Rupturas')

    return fig

---

---

Cruzamentos

In [ ]:
def cruzamentos_por_time(json, times_id):
    dict_cruzamentos_time_x = {}
    dict_cruzamentos_time_y = {}

    for id in times_id:
        n_cruzamento = 0
        cruzamentos = json["time"][f"{id}"]["rupturas"]

        for cruzamento in cruzamentos:
            n_cruzamento += 1

            instante_cruzamento = cruzamento["instante_cruzamento"]

            # Verifica em qual tempo do jogo o cruzamento ocorreu
            lista_instante_cruzamento = instante_cruzamento.split(":")
            segundos = minutos_segundos(lista_instante_cruzamento)
            if segundos <= 2700:
                tempo = "1º T"
            else:
                tempo = "2º T"

            # Informações sobre o cruzamento
            infos = {
                "logo": json["time"][f"{id}"]["logo"],
                "instante_cruzamento": cruzamento["instante_cruzamento"],
                "tempo": tempo,
                "desfecho": cruzamento["desfecho"],
                "zona": cruzamento["zona"],
                "atacantes": cruzamento["nome_jogadores_time_cruzando"],
                "defensores": cruzamento["nome_jogadores_time_defendendo"],
            }

            # Adiciona as informações no dicionário
            if id == 1:
                dict_cruzamentos_time_x[f"cruzamento_{n_cruzamento}"] = infos
            else:
                dict_cruzamentos_time_y[f"cruzamento_{n_cruzamento}"] = infos

    return dict_cruzamentos_time_x, dict_cruzamentos_time_y

---

Frequência nas sub-zonas

In [ ]:
def frequencia_subzonas(json, times_id):
    dict_zonas_time_x = {}
    dict_zonas_time_y = {}

    for id in times_id:
        cruzamentos = json["time"][f"{id}"]["rupturas"]

        if id == times_id[0]:
            for zona in cruzamentos:
                if zona["zona"] not in dict_zonas_time_x:
                    dict_zonas_time_x[zona["zona"]] = 1
                else:
                    dict_zonas_time_x[zona["zona"]] += 1
        else:
            for zona in cruzamentos:
                if zona["zona"] not in dict_zonas_time_y:
                    dict_zonas_time_y[zona["zona"]] = 1
                else:
                    dict_zonas_time_y[zona["zona"]] += 1

    return dict_zonas_time_x, dict_zonas_time_y

---

Gráfico de desfechos dos cruzamentos

In [ ]:
def grafico_desfechos_cruzamentos(json, time_id):
    desfecho = json["time"][f"{time_id}"]["rupturas"]
    desfechos = []
    n_cruzamentos = []

    for ruptura in desfecho:
        if ruptura["desfecho"] not in desfechos:
            desfechos.append(ruptura["desfecho"])
            n_cruzamentos.append(1)
        else:
            n_cruzamentos[desfechos.index(ruptura["desfecho"])] += 1

    # Fazendo o gráfico com o Plotly
    df = pd.DataFrame(dict(
        desfecho=desfechos,
        n_cruzamentos=n_cruzamentos
        ))
    
    fig = px.pie(df, values='n_cruzamentos', names='desfecho', title='Desfechos dos Cruzamentos')

    return fig